# API Documentation for Studysafe-Trace
***

## Content
#### 1.  Dependencies
#### 2.  Introductions
#### 3.  View, insert, modify venue record
#### 4.  View, insert, modify studnet record
#### 5.  Insert/Delete Enter/Exit Record
#### 6.  Better Representation
***

## Dependencies


### Python version and packages
Python version~=3.9

Dependencies for the project includes the following: 
* django = "~=4.0"
* gunicorn = "\*"
* requests = "\*"
* environs = {extras = ["django"], version = "\*"}
* djangorestframework = "\*"
* pandas = "\*"
* numpy = "\*"
* grpcio = "\*"
* ipython = "\*"
* django-extensions = "\*"
* notebook = "\*"
* pyyaml = "\*"
* uritemplate = "\*"

### To fast install dependencies in virtual environment: 
1. Make sure pipenv package is installed in python. If pipenv is not availeble, run the following code first: 
```
pip install pipenv
```
2. 
```
pipenv install
pipenv shell
```

## Introduction

***

##### Studysafe Trace is an API designed for maintaining records of the times at which members of HKU enter and exit enclosed public venues such as classrooms and lecture theatres on campus.
##### Main consumer of this API are devices across HKU campus and Studysafe Trace, a product for conveniently tracking the visited venue and close contacts of a infectious student. 
##### It maintains Venue and Member records of HKU, and can be used to record every enter/exit record made by a HKU Member to/from monitored Venues. The database supports the needs of StudySafe Trace by identifying Member and Venue, and the date and time of entry or exit

##### The endpoint for Studysafe Trace is deployed at: https://shielded-tor-28383.herokuapp.com/studysafe_core/api_trace. In order to use the endpoint type of command and relevant information must be provided through data parameter in the request. Details will be specified in the following sections. 
##### Details of this documentation can be found under source code folder, in api_documentation_studysafe_core.ipynb

## View, insert, modify venue record

***

##### Studysafe Trace API mainly takes in two inputs, type(required) and content(optional). Content input should be a list of dictionaries carrying the required data fields for the command. Type and content should be wrapped inside a dictionary, and passed to the request through data parameter as json dumped data. 
### Inputs: 
#### Type Parameter: 
modifications for venue records: ['create_venue','list_all_venue','view_venue','modify_venue','delete_venue']

modifications for student records: ['create_student','delete_student','list_all_students','search_student']

modifications for enter/exit records: ['create','delete']
#### Content Parameter: 
Content parameter should be a list of dictionaries carrying information needed in each command, and should vary according to the command type. Details would be illustrated in later sections. 

#### **Studysafe Trace allows for simultaneously carrying out multiple commands of the same type in one request, by including multiple dictionaries in the content list.**

Examples: 
```
{"type":"create_student","content":[{"hkuid":"29706","name":"Chan, Tai Man"},{"hkuid":"3035535066","name":"Chandler Bing"}]}
{"type":"modify_venue","content":[{"venue_code":"TT403","location":"","type":"","capacity":"120"}]}
{"type":"list_all_students"}

```
### Outputs: 
The API would return a json type return that requires json loading to decode. The recommanded way of loading the content of return is: 
```
json.loads(response.content.decode('utf-8'))
```
After decoding, the return value is a dictionary carrying state of command and relevant information. 
```
[{'state': 'success',
  'cmd': 'list_all',
  'data': [{'venue_code': 'CPD-2.58',
    'location': 'Centennial Campus, Central Podium Levels - Two',
    'type': 'LT',
    'capacity': 100},
   {'venue_code': 'KK101',
    'location': 'K.K. Leung Building, Main Campus',
    'type': 'LT',
    'capacity': 120},
   {'venue_code': 'MB135',
    'location': 'Main Building, Main Campus',
    'type': 'TR',
    'capacity': 14},
   {'venue_code': 'TT403',
    'location': 'Main Campus, T.T. Tsui Building',
    'type': 'CR',
    'capacity': 120}]}]
```

### View All Venue records-list_all_venue
***
**Content input not required**

**Reponse**\
Reponse Code 200: Success\
Reponse Content: State, Command and Summary of data requested

In [6]:
#import modules for demonstration purpose
import requests
import datetime
import json
import random
import pandas as pd

In [44]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"list_all_venue"}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content)

[{'state': 'success',
  'cmd': 'list_all',
  'data': [{'venue_code': 'CPD-2.58',
    'location': 'Centennial Campus, Central Podium Levels - Two',
    'type': 'LT',
    'capacity': 100},
   {'venue_code': 'KK101',
    'location': 'K.K. Leung Building, Main Campus',
    'type': 'LT',
    'capacity': 120},
   {'venue_code': 'MB135',
    'location': 'Main Building, Main Campus',
    'type': 'TR',
    'capacity': 14},
   {'venue_code': 'TT403',
    'location': 'Main Campus, T.T. Tsui Building',
    'type': 'CR',
    'capacity': 120}]}]

### View information about one venue: view_venue
***
**Parameters in Content:**\
**Venue Code**: \
type: string\
requirement: maximum length 20

**Reponse**\
Reponse Code 200: Success\
Reponse Content: State and Data requested

In [35]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"view_venue","content":[{"venue_code":"TT403"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'success',
  'data': [{'venue_code': 'TT403',
    'location': 'Main Campus, T.T. Tsui Building',
    'type': 'CR',
    'capacity': 120}]}]

### Modify information about one venue: modify_venue
***
**Parameters in Content:**\
**If one parameter is not required to be modified, can be set to be empty string "" to avoid mofification.** \
**Venue Code**: \
type: string\
requirement: maximum length 20\
**Location**: \
type: string\
requirement: maximum length 150\
**type**: \
type: string\
requirement: can only be in ['LT','CR','TR']\
**capacity**: \
type: string\
requirement: string of positive integer

**Reponse**\
Reponse Code 200: Success\
Reponse Content: Summary of the command executed

In [43]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"modify_venue","content":[{"venue_code":"TT403","location":"","type":"","capacity":"120"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': "Successfully modified venue with parameters {'venue_code': 'TT403', 'capacity': '120'}"}]

### Create record of a venue: create_venue
***
**Parameters in Content:**\
**Venue Code**: \
type: string\
requirement: maximum length 20\
**Location**: \
type: string\
requirement: maximum length 150\
**type**: \
type: string\
requirement: can only be in ['LT','CR','TR']\
**capacity**: \
type: string\
requirement: string of positive integer

**Reponse**\
Reponse Code 200: Success\
Reponse Content: Summary of the command executed

In [40]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"create_venue","content":[{"venue_code":"TT404","location":"Main Campus, T.T. Tsui Building","type":"CR","capacity":"120"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'successfully created item with the following record: TT404//Main Campus, T.T. Tsui Building//CR//120'}]

### Delete record of one venue: delete_venue
***
**Parameters in Content:**\
**Venue Code**: \
type: string\
requirement: maximum length 20

**Reponse**\
Reponse Code 200: Success
Reponse Content: Summary of the command executed

In [41]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"delete_venue","content":[{"venue_code":"TT404"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'Successfully Deleted venue TT404'}]

## View, insert, modify student record

### View All Student records-list_all_student
***
**Content input not required**

**Reponse**\
Reponse Code 200: Success\
Reponse Content: state, command, and data requested

In [45]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"list_all_students"}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'Success',
  'cmd': 'list_all',
  'data': [{'hkuid': '29705', 'name': 'Cheung, Ka Fai'},
   {'hkuid': '3023776542', 'name': 'Loo, Chi Nan'},
   {'hkuid': '3024932785', 'name': 'Chai, Wun Ching'},
   {'hkuid': '3025258327', 'name': 'Yip, Yau Shing'},
   {'hkuid': '3025704501', 'name': 'Lok, Wing Ching'},
   {'hkuid': '32154', 'name': 'Chan, Weng Yip'},
   {'hkuid': '33623', 'name': 'Lai, Cheuk Man'}]}]

### View information about one student: search_student
***
**Parameters in Content:**\
**hkuid**: \
type: string\
requirement: maximum length 10

**Reponse**\
Reponse Code 200: Success\
Reponse Content: state, command, and information of the command executed

In [46]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"search_student","content":[{"hkuid":"29705"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'Success',
  'cmd': 'search_student',
  'data': [{'hkuid': '29705', 'name': 'Cheung, Ka Fai'}]}]

### Update information about one student: update_student
***
**Parameters in Content:**\
**hkuid**: \
type: string\
requirement: maximum length 10\
**name**: \
type: string\
requirement: maximum length 150

**Reponse**\
Reponse Code 200: Success\
Reponse Content: state, command, and information of the command executed

In [47]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"update_student","content":[{"hkuid":"29705","name":"Cheung, Ka Fai"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'HKU member record updated successfully with new name Cheung, Ka Fai'}]

### Create record of one student: create_student
***
**Parameters in Content:**\
**hkuid**: \
type: string\
requirement: maximum length 10\
**name**: \
type: string\
requirement: maximum length 150

**Reponse**\
Reponse Code 200: Success
Reponse Content: Summary of the command executed

In [48]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"create_student","content":[{"hkuid":"29706","name":"Chan, Tai Man"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'successfully created item with the following record: 29706//Chan, Tai Man'}]

### Delete record of one student: delete_student
***
**Parameters in Content:**\
**hkuid**: \
type: string\
requirement: maximum length 10\
**Reponse**\
Reponse Code 200: Success\
Reponse Content: Summary of the command executed

In [49]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"delete_student","content":[{"hkuid":"29706"}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'HKU member record deleted successfully: 29706'}]

## Insert/Delete Enter/Exit Record

### Create record of one enter/exit-create
***
**Parameters in Content:**\
**hkuid**: \
type: string\
requirement: maximum length 10\
**Venue Code**: \
type: string\
requirement: maximum length 20\
**type**: \
type: string\
requirement: type must be in ["EN","EX"], where "EN" stands for Enter and "EX" stands for Exit\
**datetime**: \
type: string\
requirement: datetime must be in the form "%Y-%m-%d %H:%M:%S"

**Reponse**\
Reponse Code 200: Success\
Reponse Content: Summary of the command executed

In [65]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
enter_time=datetime.datetime(2022,3,1,0,0)
hkuid='32154'
venue_code='CPD-2.58'
params={'type':'create','content':[{'hkuid':hkuid,'venue_code':venue_code,'datetime':enter_time.strftime('%Y-%m-%d %H:%M:%S'),'type':'EN'}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'Successfully Created recordNone',
  'cmd': 'create',
  'hkuid': '32154',
  'recordid': 37,
  'venue': 'CPD-2.58',
  'datetime': '2022-03-01 00:00:00'}]

### Delete record of one enter/exit-delete
***
**Parameters in Content:**\
**recordid**: \
type: integer\
requirement: NA

**Reponse**\
Reponse Code 200: Success\
Reponse Content: Summary of the command executed

In [7]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={'type':'delete','content':[{'recordid':10}]}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
json.loads(response.content.decode('utf-8'))

[{'state': 'Successfully deleted', 'cmd': 'delete', 'recordid': 10}]

## Better Representation

May use the following defined function for convenient manipulation of data retrieved

In [15]:
def display_detail(response,return_df=False):
    raw=json.loads(response.content.decode('utf-8'))
    for i in range(len(raw)):
        try:
            if not return_df:
                display(pd.DataFrame(raw[i]['data']))
        except:
            display(raw[i])
    if return_df:
        try:
            return pd.DataFrame(raw[0]['data'])
        except Exception as e:
            display(str(e))
            

In [16]:
core_path="https://shielded-tor-28383.herokuapp.com/studysafe_core/api_device"
params={"type":"list_all_venue"}
response=requests.post(core_path,data=json.dumps(params),headers = {'content-type':'application/json'})
venue_df=display_detail(response,return_df=True)
venue_df

,venue_code,location,type,capacity
0,CPD-2.58,"Centennial Campus, Central Podium Levels - Two",LT,100
1,KK101,"K.K. Leung Building, Main Campus",LT,120
2,MB135,"Main Building, Main Campus",TR,14
3,TT403,"Main Campus, T.T. Tsui Building",CR,120
